In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

,カテゴリー,商品コード,商品名,売上日,単価,数量,原価
0,食品,1001,りんご,2023-01-01,200,50,120
1,食品,1002,バナナ,2023-01-01,150,100,80
2,食品,1003,牛乳,2023-01-02,180,80,100
3,衣服,2001,Tシャツ,2023-01-02,1500,20,800
4,衣服,2002,ジーンズ,2023-01-03,5000,10,2500


In [3]:
# 2. データをLLM用にテキスト形式に変換
# データフレーム全体を文字列に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

売上データ:
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [4]:
# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

売上データの傾向を分析する際には、以下のいくつかの観点から考察が可能です。

### 1. **売上総額の分析**
   - 売上総額を計算し、売上の傾向やピーク時期を把握します。
   - もし売上が月ごとに別れるなら、月ごとの売上推移グラフを作成することで、特定の季節にどのような商品が売れるかを視覚化できます。

### 2. **カテゴリ別の売上分析**
   - 売上データをカテゴリ別に集計し、どのカテゴリが最も売上貢献度が高いのかを確認します。たとえば、食品、衣服、日用品のカテゴリの比較が考えられます。
   - 各カテゴリ内のトレンド（例えば、特定の商品が他の商品に比べてどれほど売上を上げているか）を分析します。

### 3. **商品コード・商品名別のパフォーマンス**
   - 各商品ごとの売上げ、数量、利益（売上額 - 原価）を計算し、最も利益をもたらしている商品を特定します。
   - 強いパフォーマンスを示している商品とそうでない商品を比較することで、商品の陳列やプロモーション戦略を改善する手助けができます。

### 4. **トレンド分析**
   - 売上の成長率や減少率を分析し、特に急増または急減した商品の情報を探求します。
   - 単価と数量の関係を考慮し、どの要因が売上に最も影響を与えているかを隠れた要因として把握します。

### 5. **利益率の分析**
   - 各商品の利益率（（単価 - 原価）/ 単価）を算出し、どの商品が高い利益を生んでいるかを評価します。
   - 利益性の高い商品を強化するため、マーケティング戦略を再考することができます。

### 6. **時間帯・季節ごとの傾向**
   - 売上日をもとに、季節ごとや時間帯ごとの売上分析を行います。たとえば、特定の季節に需要が高まる商品を特定できます。
   - 売上のピークがいつであるのか（例: 年末年始、夏のセール等）を把握し、事前にマーケティングキャンペーンを計画。

### 7. **リピート購入・顧客分析**
   - 顧客情報があれば、リピート購入の傾向を分析し、どの顧客セグメントがリピーターであるかを特定します。
   - それに基づいて、特別なオファーやロイヤリティプログラムを考案することが可能です。

これらの観点から、売上データを詳しく

In [5]:
# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)

                                                   結果
0               売上データの傾向を分析する際には、以下のいくつかの観点から考察が可能です。
1                                                    
2                                  ### 1. **売上総額の分析**
3                       - 売上総額を計算し、売上の傾向やピーク時期を把握します。
4      - もし売上が月ごとに別れるなら、月ごとの売上推移グラフを作成することで、特定の季節に...
5                                                    
6                               ### 2. **カテゴリ別の売上分析**
7      - 売上データをカテゴリ別に集計し、どのカテゴリが最も売上貢献度が高いのかを確認します...
8      - 各カテゴリ内のトレンド（例えば、特定の商品が他の商品に比べてどれほど売上を上げてい...
9                                                    
10                      ### 3. **商品コード・商品名別のパフォーマンス**
11     - 各商品ごとの売上げ、数量、利益（売上額 - 原価）を計算し、最も利益をもたらしてい...
12     - 強いパフォーマンスを示している商品とそうでない商品を比較することで、商品の陳列やプ...
13                                                   
14                                  ### 4. **トレンド分析**
15           - 売上の成長率や減少率を分析し、特に急増または急減した商品の情報を探求します。
16     - 単価と数量の関係を考慮し、どの要因が売上に最も影響を与えているかを隠れた要因として...
17                          

In [6]:
# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

In [7]:
# ワークフロー化
print("処理を開始します。")

# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')

# 2. データをLLM用にテキスト形式に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"

# 3. OpenAI APIの呼び出し
# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"
# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])

# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

print("Excelファイルに分析結果を保存しました。")

処理を開始します。
Excelファイルに分析結果を保存しました。
